In [2]:
!pip install shap
!pip install pyod

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 575.9/575.9 KB 8.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.7/147.7 KB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyod: filename=pyod-1.0.7-py3-none-any.whl size=181101 sha256=03127a7f5d370bb3f464ce1c2d1b48476ae751d41e8e4191e6d903cb09891bdd
  Stored in directory: /root/.cache/pip/wheels/f7/e2/c1/1c7fd8b261e72411f6509afb429c84532e40ddcd96074473f4
Successfully built pyod


In [27]:
#import문은 최상단인 이곳으로...
import numpy as np
import pandas as pd
import io
from collections import Counter
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from pyod.models.iforest import IForest
from pyod.models.auto_encoder import AutoEncoder
from pyod.models.mcd import MCD
from pyod.models.lof import LOF
from pyod.models.ocsvm import OCSVM
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler ,RobustScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import LocalOutlierFactor
from sklearn.manifold import TSNE
from sklearn.ensemble import IsolationForest
from sklearn.covariance import EllipticEnvelope
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, classification_report, accuracy_score
from keras.layers import Input, Dense
from keras.models import Model, Sequential
from keras import regularizers
#AutoEncoder standart
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,losses,models
from tensorflow.keras.models import Model, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
#plotly standard
import plotly.offline as pyo
import plotly.graph_objs as go
import plotly.express as px
import plotly.io as pio
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
from IPython.display import Image

plt.rc('font', family = 'DejaVu Sans')
warnings.filterwarnings('ignore')
plt.rcParams['figure.figsize']=(20,10)
pyo.init_notebook_mode()

data = pd.read_csv("/content/drive/MyDrive/ColabNotebooks/DataBase/BISTelligen_Project_Data.csv")
pd_copy = data.copy()

data[data['Vibration_RMS2']<0]

prev_sum=data[data['key']==4].loc[942:981]['Vibration_RMS2'].sum()
after_sum=data[data['key']==4].loc[983:1022]['Vibration_RMS2'].sum()
pd_copy.loc[data['Vibration_RMS2']<0,'Vibration_RMS2']=(prev_sum+after_sum)/80
data

key_sum = 0
total = 0
data = pd_copy
for i in range(len(data)) :
  total += 1
  key_sum += 1
  if (i == len(data) - 1) :
    print('Key' + str(int(data.iloc[i]['key'])) + '의 개수 : ' + str(key_sum))
    print('전체 데이터셋 행개수 : ' + str(total))
    break
  else :
    if (data.iloc[i]['key'] == data.iloc[i + 1]['key']) :
      continue
    else :
      print('Key' + str(int(data.iloc[i]['key'])) + '의 개수 : ' + str(key_sum))
      key_sum = 0

def splitdataset(data, num, start_key=1, end_key=6):
    correct_dataframe = df=pd.DataFrame()
    for key in range(start_key,end_key+1):
        new_df = data[data['key']==key][:num]
        correct_dataframe = pd.concat([correct_dataframe,new_df])
    #정상치만 모은 데이터프레임
    normal_index=correct_dataframe.index
    correct_dataframe.reset_index(drop=True, inplace=True)
    correct_dataframe.head()
    return correct_dataframe,normal_index

def huepercentage(data, num, percentage, isae) :
  rightsplitlist = [num, num + 330, num + 330 + 300, num + 330 + 300 + 308, num + 330 + 300 + 308 + 326, num + 330 + 300 + 308 + 326 + 250]
  leftsplitlist = [0, 330, 330 + 300, 330 + 300 + 308, 330 + 300 + 308 + 326, 330 + 300 + 308 + 326 + 250]
  pc = (100 - percentage) * 0.01
  data['hue'] = 0
  for i in range(2):
    if i == 0 :
      if isae == True :
        for j in range(6) :
          data.loc[
            leftsplitlist[j]:int((leftsplitlist[j] + num*pc))
            , :]['hue'] = 0
          data.loc[
            int((leftsplitlist[j] + num*pc)):(leftsplitlist[j] + num)
            , :]['hue'] = 1
      else :
        for j in range(6) :
          data.loc[
            leftsplitlist[j]:int((leftsplitlist[j] + num*pc*0.6))
            , :]['hue'] = 0
          data.loc[
            int((leftsplitlist[j] + num*pc*0.6)):int((leftsplitlist[j] + num*pc*0.8))
            , :]['hue'] = 1
          data.loc[
            int((leftsplitlist[j] + num*pc*0.8)):int((leftsplitlist[j] + num*pc))
            , :]['hue'] = 2
          data.loc[
            int((leftsplitlist[j] + num*pc)):(leftsplitlist[j] + num)
            , :]['hue'] = 3
    else:
      if isae == True :
        for j in range(6) :
          data.loc[
            rightsplitlist[j]:int((rightsplitlist[j] + num*pc))
            , :]['hue'] = 0
          data.loc[
            int((rightsplitlist[j] + num*pc)):(rightsplitlist[j] + num)
            , :]['hue'] = 1
      else :
        for j in range(6) :
          data.loc[
            rightsplitlist[j]:int((rightsplitlist[j] + num*pc*0.6))
            , :]['hue'] = 0
          data.loc[
            int((rightsplitlist[j] + num*pc*0.6)):int((rightsplitlist[j] + num*pc*0.8))
            , :]['hue'] = 1
          data.loc[
            int((rightsplitlist[j] + num*pc*0.8)):int((rightsplitlist[j] + num*pc))
            , :]['hue'] = 2
          data.loc[
            int((rightsplitlist[j] + num*pc)):(rightsplitlist[j] + num)
            , :]['hue'] = 3
  return data

def hue_count(data) :
  summary_hue = data.hue.value_counts(ascending=True).to_frame().reset_index()
  summary_hue['percent'] = summary_hue["hue"].apply(lambda x : round(100*float(x) / len(data), 2))
  summary_hue = summary_hue.rename(columns = {"index" : "Target", "hue" : "Count"})
  return summary_hue

def maketraintestsplit(data, train_df) :
  train_data=pd.DataFrame(scaler.fit_transform(train_df.drop(['key'],axis=1)),columns=train_df.columns[1:])
  test_data=data.drop(['key'],axis=1)
  test_data=pd.DataFrame(scaler.fit_transform(test_data),columns=test_data.columns)
  return train_data, test_data

splitdata = data.copy()
data_01 = huepercentage(splitdata, 120, 20, True).copy()
hue_count(data_01)

data_0123 = huepercentage(splitdata, 120, 20, False).copy()
hue_count(data_0123)

scaler=RobustScaler()
#scaler=MinMaxScaler()
total_df, total_index = splitdataset(data, 120)
'''
  대부분의 분석은 train_data, test_data 사용하시면 되고,
  시각화 사용 시 아래 두개의 데이터를 사용하시면 됩니다.
'''
init_train_data, init_test_data = total_df.drop(['key'],axis=1), data.drop(['key'],axis=1)
train_data, test_data = maketraintestsplit(data, total_df)
train_data1, test_data1 = maketraintestsplit(data_01, total_df)
train_data2, test_data2 = maketraintestsplit(data_0123, total_df)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
import copy  #전체에서 각 feature를 빼서 decision function을 비교


model = IForest(contamination=0.01,random_state=42,max_samples=300,n_estimators=200)
def novelty_contribution(model, train_data, test_data, anomaly_count):
    model.fit(train_data)
    new_df=pd.DataFrame()
    model_anomaly_score=(model.decision_function(test_data))#전체 데이터에 대한 novelty score
    model_df=copy.deepcopy(test_data)
    model_df['anomaly_score']=model_anomaly_score
    model_df.sort_values(by='anomaly_score', axis=0, ascending=False, inplace=True)
    model_df_index=model_df[model_df['anomaly_score']>0].index
    model_df_index= model_df.iloc[:anomaly_count].index
    
    
    for i in range(7):
        model_train=train_data.drop(train_data.columns[i],axis=1) #각 feature의 영향력을 확인하기 위해 해당 각 feature를 drop한 모델
        model_test=test_data.drop(test_data.columns[i],axis=1)
        model.fit(model_train)
        model_anomaly_score_feature=(model.decision_function(model_test)) #각 feature가 빠진 모델 novelty score
        model_anomaly_subset_score=abs(model_anomaly_score-model_anomaly_score_feature)
        new_df[str(test_data.columns[i])]=model_anomaly_subset_score

    new_df=new_df.iloc[model_df_index]
    return new_df
        
   
novelty_contribution_df=novelty_contribution(model, train_data, test_data, 20)
novelty_contribution_df


,Pressure,Power1,Power2,Vibration_Peak1,Vibration_RMS1,Vibration_Peak2,Vibration_RMS2
627,0.034681,0.024089,0.020163,0.020312,0.032668,0.015198,0.005674
329,0.049431,0.023088,0.014941,0.022284,0.033542,0.034425,0.001987
629,0.036156,0.027226,0.001387,0.020790,0.015999,0.001122,0.004469
1258,0.021760,0.012173,0.008525,0.000463,0.003289,0.005049,0.035286
1255,0.031488,0.014252,0.004974,0.005393,0.012108,0.020903,0.028031
1256,0.037342,0.017423,0.001561,0.004934,0.020154,0.014592,0.028958
936,0.039143,0.033192,0.008885,0.029443,0.010690,0.009849,0.023629
323,0.047303,0.027266,0.004632,0.006050,0.032628,0.013576,0.000250
625,0.053422,0.039845,0.003632,0.037688,0.015920,0.005815,0.019008
628,0.016890,0.017913,0.005581,0.005542,0.016027,0.009525,0.020407


In [28]:
input_dim=train_data.shape[1]
tf.random.set_seed(2)
initializer = tf.keras.initializers.HeNormal(seed=2)
momentum = 0.9
early_stopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=10)
check_point = ModelCheckpoint('best_ae_model.h5',monitor='val_loss',mode='min',save_best_only=True)


encoder=models.Sequential([
    #input layer
    layers.InputLayer(input_shape=input_dim),

    layers.Dense(64,kernel_initializer=initializer),
    layers.BatchNormalization(momentum=momentum),
    layers.ReLU(),
    
    layers.Dense(32,kernel_initializer=initializer),
    layers.BatchNormalization(momentum=momentum),
    layers.ReLU(),
    
    layers.Dense(16,kernel_initializer=initializer),
    layers.BatchNormalization(momentum=momentum),
    layers.ReLU(),
    
    layers.Dense(8,kernel_initializer=initializer),
    layers.BatchNormalization(momentum=momentum),
    layers.ReLU(),
    
    layers.Dense(4,kernel_initializer=initializer),
    layers.BatchNormalization(momentum=momentum),
    layers.ReLU(),
])

decoder=models.Sequential([ 
    
    layers.Dense(4,kernel_initializer=initializer),
    layers.BatchNormalization(momentum=momentum),
    layers.ReLU(),
    
    layers.Dense(8,kernel_initializer=initializer),
    layers.BatchNormalization(momentum=momentum),
    layers.ReLU(),
    
    layers.Dense(16,kernel_initializer=initializer),
    layers.BatchNormalization(momentum=momentum),
    layers.ReLU(),
    
    layers.Dense(32,kernel_initializer=initializer),
    layers.BatchNormalization(momentum=momentum),
    layers.ReLU(),
    
    layers.Dense(64,kernel_initializer=initializer),
    layers.BatchNormalization(momentum=momentum),
    layers.ReLU(),
    
    #output layer
    layers.Dense(input_dim,kernel_initializer=initializer),
])

model_ae=models.Sequential([
    encoder,decoder
])

model_ae.compile(optimizer=keras.optimizers.Adam(learning_rate = 0.03), 
                    loss=keras.losses.MeanSquaredError())

history = model_ae.fit(
    train_data,train_data,
    shuffle=True,
    epochs=100,
    batch_size=128,
    validation_split = 0.3,
    callbacks = [early_stopping, check_point]
)

Epoch 1/100
4/4 [==============================] - 4s 246ms/step - loss: 1.0429 - val_loss: 0.5712
Epoch 2/100
4/4 [==============================] - 0s 30ms/step - loss: 0.5366 - val_loss: 0.8177
Epoch 3/100
4/4 [==============================] - 0s 30ms/step - loss: 0.4662 - val_loss: 0.7883
Epoch 4/100
4/4 [==============================] - 0s 84ms/step - loss: 0.4084 - val_loss: 0.4879
Epoch 5/100
4/4 [==============================] - 0s 35ms/step - loss: 0.3647 - val_loss: 0.7025
Epoch 6/100
4/4 [==============================] - 0s 68ms/step - loss: 0.3287 - val_loss: 0.4857
Epoch 7/100
4/4 [==============================] - 0s 45ms/step - loss: 0.3084 - val_loss: 0.4094
Epoch 8/100
4/4 [==============================] - 0s 46ms/step - loss: 0.2988 - val_loss: 0.3630
Epoch 9/100
4/4 [==============================] - 0s 51ms/step - loss: 0.2822 - val_loss: 0.3586
Epoch 10/100
4/4 [==============================] - 0s 47ms/step - loss: 0.2672 - val_loss: 0.3312
Epoch 11/100
4/4 [

In [ ]:
model_ae.predict(test_data)[:, 0].shape

56/56 [==============================] - 0s 2ms/step


(1764,)

In [30]:
import copy  #decision_function을 하나의 예측값으로 이용해 shap_value를 구한 것
import shap

model_iforest = IForest(contamination=0.01,random_state=42,max_samples=300,n_estimators=200)
model_mcd = MCD(support_fraction=0.8,contamination=0.01,random_state=42)
model_lof = LOF(contamination=0.01,novelty=True)
model_ocsvm = OCSVM(nu=0.01)
class XAIModels:

    autoencoder = None
    
    
    
    def get_top_anomaly_to_explain(self, test_data, num_anomalies):
        predictions = self.autoencoder.predict(test_data)
        square_errors = np.power(test_data - predictions, 2)
        mse_series = pd.Series(np.mean(square_errors, axis=1))


        most_anomal_trx = mse_series.sort_values(ascending=False)
        columns = ["id", "mse_all_columns"]
        columns.extend(["squared_error_" + x for x in list(test_data.columns)])
        items = []
        for x in most_anomal_trx.items():
            item = [x[0], x[1]]
            item.extend(square_errors.loc[x[0]])
            items.append(item)

        df_anomalies = pd.DataFrame(items, columns=columns)
        df_anomalies.set_index('id', inplace=True)

        top_anomalies_to_explain = df_anomalies.head(num_anomalies).index
        return top_anomalies_to_explain

    def get_errors_df_per_record(self, data):
        

        prediction = self.autoencoder.predict(np.array([[data]])[0])[0]
        square_errors = np.power(data - prediction, 2)
        errors_df = pd.DataFrame({'col_name': square_errors.index, 'err': square_errors}).reset_index(drop=True)
        errors_df.sort_values(by='err', ascending=False, inplace=True)
        return errors_df

    def func_predict_feature(self, data):

        record_prediction = self.autoencoder.predict(data)[:, 0]
        return record_prediction


    def novelty_contribution(self, model, train_data, test_data, anomaly_count):
        
        
        
        self.autoencoder = model
        if type(model)==Sequential:
        
            top_records_to_explain = self.get_top_anomaly_to_explain(test_data, anomaly_count)
            new_df=pd.DataFrame()
            count=0

            for record_idx in top_records_to_explain:
            
                record_to_explain = test_data.loc[record_idx]

                df_err = self.get_errors_df_per_record(record_to_explain)
            
            
                df_top_err = df_err.head(1)
            
            
                shap_values_new_df=pd.DataFrame()
            

             
                explainer = shap.KernelExplainer(self.func_predict_feature, train_data.head(100).values)
                shap_values = explainer.shap_values(record_to_explain, nsamples='auto')
                shap_values=abs(shap_values)     
              
                shap_values_df=pd.DataFrame(shap_values)
                if count==0:
                    new_df=shap_values_df
                    count+=1
                else:
                    new_df= pd.concat([new_df,shap_values_df], axis=1 )
                    

            
            
            top_records_to_explain=top_records_to_explain.to_list()
            new_df=new_df.T
            new_df.index=top_records_to_explain
            new_df.columns= test_data.columns
            
            return new_df

            

            
        else:
            model.fit(train_data)
            model_anomaly_score=(model.decision_function(test_data)) 
            model_df=copy.deepcopy(test_data)
            model_df['anomaly_score']=model_anomaly_score
            model_df.sort_values(by='anomaly_score', axis=0, ascending=False, inplace=True)
            model_df=model_df[model_df['anomaly_score']>0]

            model_df_index= model_df.iloc[:anomaly_count].index

            explainer=shap.KernelExplainer(model.decision_function, train_data.head(100))
            shap_values= explainer.shap_values(test_data.iloc[model_df_index], nsamples='auto')
            novelty_df= pd.DataFrame(abs(shap_values), index= model_df_index, columns=test_data.columns)    
            return novelty_df

    

    
    
    
        


exp_model=XAIModels()
exp_model.novelty_contribution(model_ocsvm, train_data, test_data, 10)

  0%|          | 0/10 [00:00<?, ?it/s]

,Pressure,Power1,Power2,Vibration_Peak1,Vibration_RMS1,Vibration_Peak2,Vibration_RMS2
935,0.452234,0.452168,0.004877,0.101832,0.451351,0.014011,0.000516
1263,0.447025,0.447024,0.003064,0.112874,0.446070,0.012245,0.000497
626,0.490319,0.488957,0.013590,0.031870,0.399026,0.004062,0.018481
627,0.426871,0.426854,0.030312,0.099036,0.415553,0.003244,0.036311
628,0.442735,0.442684,0.000520,0.094663,0.442614,0.015557,0.031561
629,0.421627,0.421627,0.006395,0.130435,0.421626,0.000649,0.037120
929,0.463006,0.461798,0.024553,0.019291,0.446667,0.016515,0.039380
930,0.449348,0.447146,0.024680,0.077619,0.439569,0.015919,0.015737
931,0.439727,0.439002,0.014606,0.097364,0.434897,0.015106,0.027691
932,0.478689,0.462076,0.022865,0.003293,0.474866,0.015107,0.018085


In [ ]:
!jupyter nbconvert --to html '/content/drive/MyDrive/test/이상탐지/XAI.ipynb'

[NbConvertApp] Converting notebook /content/drive/MyDrive/test/이상탐지/XAI.ipynb to html
[NbConvertApp] Writing 377962 bytes to /content/drive/MyDrive/test/이상탐지/XAI.html
